In [39]:
def hit_at_k(y_true_list, y_pred_list, k=5):
    """

    y_true_list: список списков (или множеств) релевантных документов для каждого вопроса
    y_pred_list: список списков найденных документов
    k: топ-K (по умолчанию 5)

    Возвращает среднюю долю найденных релевантных документов.
    """
    total_score = 0.0

    for true_docs, pred_docs in zip(y_true_list, y_pred_list):
        true_set = set(true_docs)
        pred_set = set(pred_docs[:k])  # берем топ-k предсказаний
        found_relevant = len(true_set & pred_set)
        total_relevant = len(true_set)
        score = found_relevant / total_relevant if total_relevant > 0 else 0
        total_score += score

    return total_score / len(y_true_list)


In [1]:
from sentence_transformers import CrossEncoder

# Модель для re-ranking
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [43]:
docs = [
    # --- Космическая тематика (релевантные) ---

    "Когда звезда с массой больше примерно 20 солнечных масс исчерпывает топливо, её ядро может сжаться в черную дыру.",
    "Гравитация в черной дыре настолько сильна, что даже свет не может покинуть её пределы.",
    "Событийный горизонт — граница, после которой невозможно вернуться из черной дыры.",
    "Черные дыры бывают разных типов: звездные, промежуточные и супермассивные.",
    "Супермассивные черные дыры находятся в центрах галактик, включая нашу Млечный Путь.",
    "Черные дыры образуются из массивных звезд после их коллапса.",
    "При падении материи на черную дыру выделяется рентгеновское излучение.",
    "Изучение движения звезд вокруг центров галактик помогает обнаружить черные дыры.",
    "Червоточины — гипотетические объекты, часто связаны с черными дырами в научной фантастике.",
    "Слияние двух черных дыр создаёт гравитационные волны.",
    "Черные дыры не 'всасывают' всё подряд, они притягивают объекты, которые находятся близко.",
    "Эффект Хокинга описывает медленное испарение черных дыр через излучение.",
    "Существование черных дыр предсказывается общей теорией относительности Эйнштейна.",
    "Некоторые черные дыры могут быть двойными системами с соседними звездами.",
    "Аккреционный диск вокруг черной дыры светится из-за нагрева вещества.",
    "Сверхмассивные черные дыры могут влиять на эволюцию своей галактики.",
    "Черные дыры — одни из самых загадочных объектов во Вселенной.",
    "Изучение черных дыр помогает понять гравитацию и квантовые процессы.",
    "Существуют теории, что внутри черных дыр могут находиться новые вселенные.",
    "Гравитационные линзы могут помочь обнаруживать черные дыры.",
    "Квазары питаются сверхмассивными черными дырами и испускают огромные количества энергии.",
    "Черные дыры образуются в результате коллапса ядра массивной звезды.",
    "Пространство и время искривляются вокруг черной дыры.",
    "Черные дыры имеют массу, заряд и вращение.",
    "Детекторы гравитационных волн, такие как LIGO, регистрируют слияния черных дыр.",
    "Галактические центры часто содержат сверхмассивные черные дыры.",
    "Излучение Хокинга — теоретический процесс медленного уменьшения массы черной дыры.",
    "Черные дыры могут образовываться после суперновых.",
    "Астрономы используют рентгеновские телескопы для изучения черных дыр.",
    "Энергия из аккреционного диска черной дыры может быть мощнее целой галактики.",
    "Некоторые теории предполагают существование микроскопических черных дыр.",
    "Вращение черной дыры влияет на форму её событийного горизонта.",
    "Образование черных дыр связано с принципами ядерного синтеза и гравитации.",
    "Излучение материи из окрестностей черной дыры создаёт джеты.",
    "Черные дыры могут ускорять объекты вблизи до околосветовых скоростей.",
    "Нейтронные звезды — потенциальные предшественники черных дыр.",
    "Слияние черных дыр создаёт характерные гравитационные волны.",
    "Радиотелескопы помогают наблюдать активные ядра галактик с черными дырами.",
    "Квазары — яркие источники энергии, связанные с черными дырами.",
    "Черные дыры меняют траектории звезд в своих системах.",
    "Аккреционный диск — яркий и горячий слой материи вокруг черной дыры.",
    "Сверхмассивные черные дыры могут достигать масс миллиардов солнц.",
    "Черные дыры являются тестами для теорий гравитации и квантовой механики.",

    # --- Остальная космическая тематика (частично нерелевантные) ---
    "Звезды образуются из облаков газа и пыли.",
    "Сверхновые — взрывы массивных звезд в конце их жизни.",
    "Планеты вращаются вокруг звезд по эллиптическим орбитам.",
    "Солнечная система состоит из Солнца, планет и малых тел.",
    "Кометы — ледяные объекты с длинными хвостами при приближении к Солнцу.",
"Астероиды — каменистые тела, вращающиеся вокруг Солнца.",
    "Млечный Путь — спиральная галактика с миллиардами звезд.",
    "Галактики бывают спиральные, эллиптические и неправильные.",
    "Туманности — облака газа и пыли в космосе.",
    "Космический микроволновый фон — следы Большого взрыва.",
    "Темная материя влияет на движение галактик, но не видна напрямую.",
    "Космический телескоп Хаббл сделал множество открытий о далеких галактиках.",
    "Планета Марс изучается роботами и орбитальными станциями.",
    "Луна влияет на приливы и отливы на Земле.",
    "Солнечные пятна — временные области интенсивного магнитного поля.",
    "Космические зонды помогают изучать внешние планеты.",
    "Экзопланеты — планеты за пределами нашей солнечной системы.",
    "Чёрная дыра в центре Млечного Пути называется Стрелец A*.",
    "Некоторые астероиды могут пересекать орбиту Земли.",
    "Метеориты — остатки космических тел, падающие на Землю.",
    "Космические лучи — высокоэнергичные частицы из космоса.",
    "Гравитация управляет движением всех тел во Вселенной.",
    "Эволюция звезд зависит от их массы и химического состава.",
    "Нейтронные звезды — остатки коллапсированных массивных звезд.",
    "Космическая экспансия ускоряется из-за тёмной энергии.",
    "Телескопы позволяют наблюдать далекие галактики и квазары.",
    "Солнечная система сформировалась около 4.6 миллиардов лет назад.",
    "Космическая станция на орбите Земли помогает изучать микрогравитацию.",
    "Марсоходы исследуют поверхность планеты и собирают образцы.",
    "Кометы состоят из льда, пыли и замёрзших газов.",
    "Астрономы используют спектроскопию для изучения химии звезд.",
    "Квазар — активное ядро далекой галактики, излучающее энергию.",
    "Галактические скопления содержат тысячи галактик.",
    "Чёрные дыры иногда образуют двойные системы с соседними звёздами.",
    "Солнце — звезда главной последовательности.",
    "Космические миссии позволяют изучать внешние планеты и их спутники.",
    "Темная энергия составляет около 70% массы Вселенной.",
    "Красное смещение помогает определять скорость удаления галактик.",
    "Световой год — единица расстояния в астрономии.",
    "Большой взрыв — теория возникновения Вселенной."
]

In [ ]:
query = "Черные дыры Как образуются ?"

In [59]:

def rank(query, docs):

  # Пары (запрос, документ)
  pairs = [(query, doc) for doc in docs]

  # Предсказание релевантности
  scores = reranker.predict(pairs)

  # Сортировка
  ranked = sorted(zip(scores, docs), reverse=True)

  # Печать топ-5
  print("Топ-5 документов после re-ranking:\n")
  y = []
  for i, (score, doc) in enumerate(ranked[:5], 1):
      #print(f"{i}. ({score:.4f}) {doc}")
      y.append(doc)
  return y


In [60]:
rank(query,docs)

Топ-5 документов после re-ranking:



["Черные дыры не 'всасывают' всё подряд, они притягивают объекты, которые находятся близко.",
 'Черные дыры образуются в результате коллапса ядра массивной звезды.',
 'Супермассивные черные дыры находятся в центрах галактик, включая нашу Млечный Путь.',
 'Черные дыры являются тестами для теорий гравитации и квантовой механики.',
 'Червоточины — гипотетические объекты, часто связаны с черными дырами в научной фантастике.']

In [64]:

# Пример данных
true_chunks = ['D2', 'D5', 'D10']
retrieved_chunks = [
    ['D1', 'D2', 'D3'],
    ['D4', 'D5', 'D6'],
    ['D7', 'D8', 'D9']
]

print("Hit@1:", hit_at_k(true_chunks, retrieved_chunks, 1))  # 0.0
print("Hit@3:", hit_at_k(true_chunks, retrieved_chunks, 3))  # 0.666...

Hit@1: 0.0
Hit@3: 0.6666666666666666
